In [27]:
import pandas as pd

In [28]:
train = pd.read_csv('../input/train.csv')
print(train.shape)
train.head()

(33126, 8)


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0


In [29]:
train['patient_id'].value_counts()

IP_4382720    115
IP_7279968    115
IP_4479736    115
IP_4938382    115
IP_0656529    114
             ... 
IP_2516168      3
IP_7471612      3
IP_9460738      3
IP_0147446      3
IP_8335299      2
Name: patient_id, Length: 2056, dtype: int64

In [30]:
train['target'].value_counts(normalize=True) * 100

0    98.237034
1     1.762966
Name: target, dtype: float64

In [31]:
test = pd.read_csv('../input/test.csv')
print(test.shape)
test.head()

(10982, 5)


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,male,70.0,NaN
1,ISIC_0052349,IP_7782715,male,40.0,lower extremity
2,ISIC_0058510,IP_7960270,female,55.0,torso
3,ISIC_0073313,IP_6375035,female,50.0,torso
4,ISIC_0073502,IP_0589375,female,45.0,lower extremity


In [35]:
train_folds = pd.read_csv('../input/train_folds_stratified.csv')
print(train_folds.shape)
train_folds.groupby('kfold')['target'].value_counts(normalize=True) * 100

(33126, 9)


kfold  target
-1.0   0         98.219674
       1          1.780326
 0.0   0         98.239141
       1          1.760859
 1.0   0         98.239141
       1          1.760859
 2.0   0         98.238846
       1          1.761154
 3.0   0         98.238846
       1          1.761154
 4.0   0         98.238846
       1          1.761154
Name: target, dtype: float64

In [36]:
train_folds = pd.read_csv('../input/train_folds_groups.csv')
print(train_folds.shape)
train_folds.groupby('kfold')['target'].value_counts(normalize=True) * 100

(33126, 9)


kfold  target
-1.0   0         98.249849
       1          1.750151
 0.0   0         98.423612
       1          1.576388
 1.0   0         98.473923
       1          1.526077
 2.0   0         98.054344
       1          1.945656
 3.0   0         97.903388
       1          2.096612
 4.0   0         98.322710
       1          1.677290
Name: target, dtype: float64